<a href="https://colab.research.google.com/github/luojie1024/TextClassification/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install jieba
!pip install gensim
import pandas as pd
import json
import jieba

Looking in indexes: http://pypi.douban.com/simple/
Looking in indexes: http://pypi.douban.com/simple/


# 0. gensim实践：

1. 读取预处理好的数据
2. 训练
3. 完事

# 1. 下载数据集

In [7]:
! wget https://storage.googleapis.com/cluebenchmark/tasks/tnews_public.zip 
! unzip tnews_public.zip

--2023-12-19 09:40:53--  https://storage.googleapis.com/cluebenchmark/tasks/tnews_public.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.66.91, 142.250.66.155, 172.217.31.27, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.66.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5123575 (4.9M) [application/zip]
Saving to: ‘tnews_public.zip’

tnews_public.zip    100%[===================>]   4.89M  9.95MB/s    in 0.5s    

2023-12-19 09:40:54 (9.95 MB/s) - ‘tnews_public.zip’ saved [5123575/5123575]

Archive:  tnews_public.zip
  inflating: dev.json                
  inflating: labels.json             
  inflating: test.json               
  inflating: test1.0.json            
  inflating: train.json              
  inflating: README.txt              


## 1.1 预处理数据

In [8]:
def get_sentence(data_file):
    # 读取数据集中的句子
    f=open('train.json','r',encoding='utf-8')
    reader = f.readlines()
    sentence=[]
    for line in reader:
        line=json.loads(line.strip())
        sentence.append(line['sentence'])
    return sentence

In [9]:
# 读取句子语料
train_sentence=get_sentence('train.json')
test_sentence=get_sentence('test.json')
dev_sentence=get_sentence('dev.json')

# 2. 载入数据

In [10]:
# 使用所有语料作为词向量训练语料
train_data=train_sentence+test_sentence+dev_sentence

In [11]:
%%time
# 分词处理
train_data=[list(jieba.cut(stentence)) for stentence in train_data]

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/g0/wqv_f7ds0m91wfk_jx58wsrc0000gn/T/jieba.cache
Loading model cost 0.306 seconds.
Prefix dict has been built successfully.


CPU times: user 4.55 s, sys: 47.4 ms, total: 4.6 s
Wall time: 4.61 s


# 3. 模型创建

Gensim中 Word2Vec 模型的期望输入是进过分词的句子列表，即是某个二维数组。这里我们暂时使用 Python 内置的数组，不过其在输入数据集较大的情况下会占用大量的 RAM。Gensim 本身只是要求能够迭代的有序句子列表，因此在工程实践中我们可以使用自定义的生成器，只在内存中保存单条语句。

## Word2Vec 参数
+ min_count

在不同大小的语料集中，我们对于基准词频的需求也是不一样的。譬如在较大的语料集中，我们希望忽略那些只出现过一两次的单词，这里我们就可以通过设置min_count参数进行控制。一般而言，合理的参数值会设置在0~100之间。

+ size

size参数主要是用来设置神经网络的层数，Word2Vec 中的默认值是设置为100层。更大的层次设置意味着更多的输入数据，不过也能提升整体的准确度，合理的设置范围为 10~数百。

+ workers

workers参数用于设置并发训练时候的线程数，不过仅当Cython安装的情况下才会起作用：

In [12]:
# 引入 word2vec
from gensim.models.word2vec import LineSentence
from gensim.models import word2vec
import gensim

# 引入日志配置
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# 构建训练

## FastText

In [13]:
from gensim.models import FastText

In [16]:
help(FastText)

Help on class FastText in module gensim.models.fasttext:

class FastText(gensim.models.word2vec.Word2Vec)
 |  FastText(sentences=None, corpus_file=None, sg=0, hs=0, vector_size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, word_ngrams=1, sample=0.001, seed=1, workers=3, min_alpha=0.0001, negative=5, ns_exponent=0.75, cbow_mean=1, hashfxn=<built-in function hash>, epochs=5, null_word=0, min_n=3, max_n=6, sorted_vocab=1, bucket=2000000, trim_rule=None, batch_words=10000, callbacks=(), max_final_vocab=None, shrink_windows=True)
 |  
 |  Method resolution order:
 |      FastText
 |      gensim.models.word2vec.Word2Vec
 |      gensim.utils.SaveLoad
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, sentences=None, corpus_file=None, sg=0, hs=0, vector_size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, word_ngrams=1, sample=0.001, seed=1, workers=3, min_alpha=0.0001, negative=5, ns_exponent=0.75, cbow_mean=1, hashfxn=<built-in functi

In [19]:
model = FastText(train_data,  window=3, min_count=1,min_n = 3 , max_n = 6)

2023-12-19 09:47:17,498 : INFO : collecting all words and their counts
2023-12-19 09:47:17,500 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-12-19 09:47:17,529 : INFO : PROGRESS: at sentence #10000, processed 129141 words, keeping 25698 word types
2023-12-19 09:47:17,549 : INFO : PROGRESS: at sentence #20000, processed 258779 words, keeping 39123 word types
2023-12-19 09:47:17,566 : INFO : PROGRESS: at sentence #30000, processed 387255 words, keeping 49324 word types
2023-12-19 09:47:17,580 : INFO : PROGRESS: at sentence #40000, processed 515598 words, keeping 57864 word types
2023-12-19 09:47:17,593 : INFO : PROGRESS: at sentence #50000, processed 644895 words, keeping 65382 word types
2023-12-19 09:47:17,605 : INFO : PROGRESS: at sentence #60000, processed 774201 words, keeping 67815 word types
2023-12-19 09:47:17,617 : INFO : PROGRESS: at sentence #70000, processed 903891 words, keeping 67815 word types
2023-12-19 09:47:17,629 : INFO : PROGRESS: at 

## skip-gram 与 CBOW.

In [23]:
# sg : {0, 1}, optional Training algorithm: 1 for skip-gram; otherwise CBOW.
model = word2vec.Word2Vec(train_data, sg=1,workers=8)

2023-12-19 09:49:29,259 : INFO : collecting all words and their counts
2023-12-19 09:49:29,260 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-12-19 09:49:29,292 : INFO : PROGRESS: at sentence #10000, processed 129141 words, keeping 25698 word types
2023-12-19 09:49:29,311 : INFO : PROGRESS: at sentence #20000, processed 258779 words, keeping 39123 word types
2023-12-19 09:49:29,327 : INFO : PROGRESS: at sentence #30000, processed 387255 words, keeping 49324 word types
2023-12-19 09:49:29,339 : INFO : PROGRESS: at sentence #40000, processed 515598 words, keeping 57864 word types
2023-12-19 09:49:29,351 : INFO : PROGRESS: at sentence #50000, processed 644895 words, keeping 65382 word types
2023-12-19 09:49:29,360 : INFO : PROGRESS: at sentence #60000, processed 774201 words, keeping 67815 word types
2023-12-19 09:49:29,369 : INFO : PROGRESS: at sentence #70000, processed 903891 words, keeping 67815 word types
2023-12-19 09:49:29,379 : INFO : PROGRESS: at 

In [21]:
help(word2vec)

Help on module gensim.models.word2vec in gensim.models:

NAME
    gensim.models.word2vec

DESCRIPTION
    Introduction
    
    This module implements the word2vec family of algorithms, using highly optimized C routines,
    data streaming and Pythonic interfaces.
    
    The word2vec algorithms include skip-gram and CBOW models, using either
    hierarchical softmax or negative sampling: `Tomas Mikolov et al: Efficient Estimation of Word Representations
    in Vector Space <https://arxiv.org/pdf/1301.3781.pdf>`_, `Tomas Mikolov et al: Distributed Representations of Words
    and Phrases and their Compositionality <https://arxiv.org/abs/1310.4546>`_.
    
    Other embeddings
    
    There are more ways to train word vectors in Gensim than just Word2Vec.
    See also :class:`~gensim.models.doc2vec.Doc2Vec`, :class:`~gensim.models.fasttext.FastText`.
    
    The training algorithms were originally ported from the C package https://code.google.com/p/word2vec/
    and extended with add

# 查找最近的词

In [30]:
model.wv.most_similar(['办公'],topn=10)

[('OA', 0.8478599190711975),
 ('商标注册', 0.831925094127655),
 ('泛微', 0.831880509853363),
 ('识别', 0.8070147633552551),
 ('试水', 0.8049625754356384),
 ('柜及', 0.8012972474098206),
 ('随心所欲', 0.7993997931480408),
 ('流媒体', 0.7984649538993835),
 ('叫车', 0.7980546951293945),
 ('安和力', 0.7939438819885254)]

# 保存模型

In [34]:
save_model_path='word2vec.model'

In [35]:
model.save(save_model_path)

2023-12-19 09:54:23,092 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'word2vec.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-12-19T09:54:23.092532', 'gensim': '4.3.2', 'python': '3.11.5 (main, Sep 11 2023, 08:31:25) [Clang 14.0.6 ]', 'platform': 'macOS-14.1.2-arm64-arm-64bit', 'event': 'saving'}
2023-12-19 09:54:23,094 : INFO : not storing attribute cum_table
2023-12-19 09:54:23,124 : INFO : saved word2vec.model


# 载入模型

In [53]:
model = word2vec.Word2Vec.load(save_model_path)

2021-04-05 06:43:03,418 : INFO : loading Word2Vec object from word2vec.model
2021-04-05 06:43:03,859 : INFO : loading wv recursively from word2vec.model.wv.* with mmap=None
2021-04-05 06:43:03,862 : INFO : setting ignored attribute vectors_norm to None
2021-04-05 06:43:03,866 : INFO : loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
2021-04-05 06:43:03,870 : INFO : loading trainables recursively from word2vec.model.trainables.* with mmap=None
2021-04-05 06:43:03,876 : INFO : setting ignored attribute cum_table to None
2021-04-05 06:43:03,877 : INFO : loaded word2vec.model


In [31]:
model.wv.most_similar(['奇瑞'],topn=10)

[('纯电', 0.8425928354263306),
 ('捷豹', 0.8233745098114014),
 ('上汽', 0.8183848857879639),
 ('智联', 0.8133119344711304),
 ('开瑞', 0.8040586709976196),
 ('X3', 0.7948211431503296),
 ('欧蓝德', 0.793942391872406),
 ('瑞虎', 0.792582631111145),
 ('T600', 0.7899853587150574),
 ('昂科威', 0.7895016670227051)]

# 参考

1. https://radimrehurek.com/gensim/models/word2vec.html 